$$\underline{\textbf{Making a Model a Simulating a Simple Pendulum}}$$

This notebook will serve as a proof-of-concept for our project, showing that we can extend this concept to a physical pendulum and later a chaotic double pendulum.  
This notebook will follow the following general lay out:  
$\textbf{Step 1}$: Developing a Mathematical Model  
$\textbf{Step 2}$: Generating Plots of Data  
$\textbf{Step 3}$: Generating a Simulation  

$$\underline{\textbf{Step 1: Developing a Mathematical Model}}$$
I would like to first get the equations of motion in polar coordinates using the Euler-Lagrange equations.  
Let me draw a picture for clarity  

<p align="center"><img src="./images/pendulum.jpg" alt="drawing" width="400"/></p>

Now we can easily define the kinetic and potential a follows  
$$T=\frac{1}{2}mL^2\dot{\theta}$$
$$V=mgL(1-\cos(\theta))$$
Which leads to the Lagrangian  
$$L=T-V=\frac{1}{2}mL^2\dot{\theta}-mgL(1-\cos(\theta))$$
The Euler-Lagrange equation gives  
$$\frac{\partial}{\partial t}\left(\frac{\partial L}{\partial \dot{\theta}}\right)-\frac{\partial L}{\delta \theta}=0$$
And we know 
$$\frac{\partial L}{\partial \dot{\theta}}=mL^2\dot{\theta}\implies\frac{\partial}{\partial t}\left(\frac{\partial L}{\partial \dot{\theta}}\right)=mL^2\ddot{\theta}$$
$$\frac{\partial L}{\delta \theta}=-mgL\sin(\theta)$$
Plugging into the Euler-Lagrange equation gives 
$$mL^2\ddot{\theta}+mgL\sin(\theta)=0$$
Cancelling the common terms (L and m) and dividing the entire equation by L gives the recognizable equation 
$$\ddot{\theta}+\frac{g}{L}\sin(\theta)=0$$